#Install Required Libraries

In [ ]:
!pip install keras tensorflow

In [3]:
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [4]:
import pandas as pd
import numpy as np
import string, os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

#Import Dataset

In [32]:
all_headlines = ["A clever monkey lived by a river crossing to an island full of fruit by hopping on a large stone", "One day, a crocodile couple wanted to catch the monkey", "The female crocodile sat on the stone, making it look much bigger", "The monkey realizing it was a trap pretended to be scared and asked the crocodile to open its mouth so he could jump in", "When the crocodile foolishly opened its mouth the monkey jumped onto its head instead using it to cross the river and escape" ,"The moral is to use your intelligence and not be foolish"]

In [33]:
print(all_headlines[:10])

['A clever monkey lived by a river crossing to an island full of fruit by hopping on a large stone', 'One day, a crocodile couple wanted to catch the monkey', 'The female crocodile sat on the stone, making it look much bigger', 'The monkey realizing it was a trap pretended to be scared and asked the crocodile to open its mouth so he could jump in', 'When the crocodile foolishly opened its mouth the monkey jumped onto its head instead using it to cross the river and escape', 'The moral is to use your intelligence and not be foolish']


#Removing Special Characters

In [34]:
import string, os
def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt
corpus = [clean_text(x) for x in all_headlines]
print(corpus[:10])

['a clever monkey lived by a river crossing to an island full of fruit by hopping on a large stone', 'one day a crocodile couple wanted to catch the monkey', 'the female crocodile sat on the stone making it look much bigger', 'the monkey realizing it was a trap pretended to be scared and asked the crocodile to open its mouth so he could jump in', 'when the crocodile foolishly opened its mouth the monkey jumped onto its head instead using it to cross the river and escape', 'the moral is to use your intelligence and not be foolish']


#Generating n-gram Sequence for training

In [36]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to a token sequence
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)
for i in inp_sequences[:50]:
  print(i)

[3, 15]
[3, 15, 4]
[3, 15, 4, 16]
[3, 15, 4, 16, 9]
[3, 15, 4, 16, 9, 3]
[3, 15, 4, 16, 9, 3, 10]
[3, 15, 4, 16, 9, 3, 10, 17]
[3, 15, 4, 16, 9, 3, 10, 17, 2]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21, 22]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21, 22, 9]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21, 22, 9, 23]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21, 22, 9, 23, 11]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21, 22, 9, 23, 11, 3]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21, 22, 9, 23, 11, 3, 24]
[3, 15, 4, 16, 9, 3, 10, 17, 2, 18, 19, 20, 21, 22, 9, 23, 11, 3, 24, 12]
[25, 26]
[25, 26, 3]
[25, 26, 3, 5]
[25, 26, 3, 5, 27]
[25, 26, 3, 5, 27, 28]
[25, 26, 3, 5, 27, 28, 2]
[25, 26, 3, 5, 27, 28, 2, 29]
[25, 26, 3, 5, 27, 28, 2, 29, 1]
[25, 26, 3, 5, 27, 28, 2, 29, 1, 4]
[1, 30]
[1, 30, 5]
[1, 30, 5, 3

#Padding the Sequences

In [37]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

#Building Model with Keras and Compiling

In [46]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(500))
    model.add(Dropout(0.1))

    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

#Fitting and Training Model

In [48]:
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

#Function to Generate Predicted Sequence

In [49]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],              maxlen=max_sequence_len-1, padding='pre')
        predicted = model(token_list, verbose=0)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == int(tf.argmax(predicted,axis=1)):
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()



#Predicting Outcome

In [60]:
print (generate_text("A clever monkey lived by a river crossing to an island ", 3, model, max_sequence_len))
print (generate_text("The monkey realizing it ", 3, model, max_sequence_len))
print (generate_text("its mouth the monkey ", 4, model, max_sequence_len))

A clever monkey lived by a river crossing to an island full of fruit
The monkey realizing it was a trap
its mouth the monkey jumped onto its head
